In [9]:
## credentail init
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/frhyme/Downloads/My Project-6f3701259a8e.json', scope)

## authorize
import gspread
gc = gspread.authorize(credentials)
## sheet file 이름을 넘겨주고 읽습니다. 
wks = gc.open("!individual_meet_report").get_worksheet(0)


In [20]:
wks = gc.open("!individual_meet_report").get_worksheet(0)

## get row values 
## 0 이 아니라 1부터 시작하는 것을 유의할 것 
for i in range(1, 3):
    row = wks.row_values(i)

## get column values 
## 마찬가지로 1부터 시작함 
for i in range(1, 2):
    col = wks.col_values(i)[:10]
    print(col)


['날짜', '2018. 2. 20', '2018. 2. 20', '2018. 2. 20', '2018. 2. 20', '2018. 2. 21', '2018. 2. 21', '2018. 2. 21', '2018. 2. 21', '2018. 2. 22']


In [ ]:
# Get all values from the first row
values_list = worksheet.row_values(1)

# Get all values from the first column
values_list = worksheet.col_values(1)
val = worksheet.acell('B1').value

# With coords
val = worksheet.cell(1, 2).value